In [0]:
# Importing Libraries

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
root_path = 'gdrive/My Drive/UCI_HAR_Dataset/'

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'gdrive/My Drive/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'gdrive/My Drive/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
import tensorflow as tf

from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [14]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [15]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [16]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [18]:
# Training the model
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 31s 4ms/step - loss: 1.3154 - acc: 0.4361 - val_loss: 1.1414 - val_acc: 0.4761
Epoch 2/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.9767 - acc: 0.5830 - val_loss: 0.9244 - val_acc: 0.5996
Epoch 3/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.8059 - acc: 0.6413 - val_loss: 0.8358 - val_acc: 0.5931
Epoch 4/30
7352/7352 [==============================] - 29s 4ms/step - loss: 0.6793 - acc: 0.6658 - val_loss: 0.7258 - val_acc: 0.6118
Epoch 5/30
7352/7352 [==============================] - 29s 4ms/step - loss: 0.6383 - acc: 0.6798 - val_loss: 0.7400 - val_acc: 0.6250
Epoch 6/30
7352/7352 [==============================] - 29s 4ms/step - loss: 0.5991 - acc: 0.6931 - val_loss: 0.6634 - val_acc: 0.6987
Epoch 7/30
7352/7352 [====================

In [19]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 509        1  ...                   0                27
SITTING                  0      416  ...                   0                 2
STANDING                 0      107  ...                   0                 1
WALKING                  0        0  ...                  18                10
WALKING_DOWNSTAIRS       0        0  ...                 410                 9
WALKING_UPSTAIRS         0        1  ...                  18               437

[6 rows x 6 columns]


In [20]:
score = model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 1s 299us/step


[0.4753849825397535, 0.9039701391245334]

1 lstm layer + 128 neurons

In [0]:
model = Sequential()
# Configuring the parameters
model.add(LSTM(128, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               70656     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
Total params: 71,430
Trainable params: 71,430
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 61s 8ms/step - loss: 1.5807 - acc: 0.3094 - val_loss: 1.4886 - val_acc: 0.3332
Epoch 2/30
7352/7352 [==============================] - 59s 8ms/step - loss: 1.3687 - acc: 0.3735 - val_loss: 1.3785 - val_acc: 0.3448
Epoch 3/30
7352/7352 [==============================] - 60s 8ms/step - loss: 1.3510 - acc: 0.3753 - val_loss: 1.3936 - val_acc: 0.3448
Epoch 4/30
7352/7352 [==============================] - 60s 8ms/step - loss: 1.3634 - acc: 0.3881 - val_loss: 1.4029 - val_acc: 0.3593
Epoch 5/30
7352/7352 [==============================] - 60s 8ms/step - loss: 1.3266 - acc: 0.4181 - val_loss: 1.4404 - val_acc: 0.3336
Epoch 6/30
7352/7352 [==============================] - 60s 8ms/step - loss: 1.3597 - acc: 0.3819 - val_loss: 1.4088 - val_acc: 0.3373
Epoch 7/30
7352/7352 [==============================] - 60s 8ms/step - loss: 1.2745 - acc: 0.4392 - val_loss: 1.2551 - val_acc: 0.4133
Epoch 8

In [0]:
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
print("===================================================================")
score

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 535        1  ...                   0                 1
SITTING                  0      305  ...                   0                 0
STANDING                 0       34  ...                   0                 0
WALKING                  0        0  ...                  39                17
WALKING_DOWNSTAIRS       0        0  ...                 398                16
WALKING_UPSTAIRS         0        0  ...                  26               409

[6 rows x 6 columns]
2947/2947 [==============================] - 2s 826us/step


[0.44404495626493334, 0.8768238887003733]

1 hidden lstm layer + 64 neurons

In [22]:
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.3))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 37s 5ms/step - loss: 1.3195 - acc: 0.4166 - val_loss: 1.3536 - val_acc: 0.3655
Epoch 2/30
7352/7352 [==============================] - 34s 5ms/step - loss: 1.2865 - acc: 0.4140 - val_loss: 1.2399 - val_acc: 0.4849
Epoch 3/30
7352/7352 [==============================] - 33s 5ms/step - loss: 1.1092 - acc: 0.5133 - val_loss: 1.0629 - val_acc: 0.5273
Epoch 4/30
7352/7352 [==============================] - 33s 5ms/step - loss: 0.9822 - acc: 0.5422 - val_loss: 0.9051 - val_acc: 0.5239
Epoch 5/30
7352/7352 [==============================] - 34s 5ms/step - loss: 0.9226 - acc: 0.5839 - val_loss: 0.8441 - val_acc: 0.6312
Epoch 6/30
7352/7352 [==============================] - 34s 5ms/step - loss: 0.7458 - acc: 0.6348 - val_loss: 0.7618 - val_acc: 0.6193
Epoch 7/30
7352/7352 [==============================] - 33s 5ms/step - loss: 0.6800 - acc: 0.6717 - val_loss: 0.7005 - val_acc: 0.6542
Epoch 8

In [24]:
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
print("===================================================================")
score

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 510        0  ...                   0                 1
SITTING                  0      410  ...                   0                 2
STANDING                 0       93  ...                   0                 4
WALKING                  0        0  ...                   8                 6
WALKING_DOWNSTAIRS       0        0  ...                 376                10
WALKING_UPSTAIRS         0        0  ...                   0               424

[6 rows x 6 columns]
2947/2947 [==============================] - 1s 448us/step


[0.4244747120754469, 0.8941296233457754]

1 hidden layer + 200 neurons

In [25]:
model = Sequential()
# Configuring the parameters
model.add(LSTM(200, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 200)               168000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 1206      
Total params: 169,206
Trainable params: 169,206
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 89s 12ms/step - loss: 1.2758 - acc: 0.4344 - val_loss: 1.2578 - val_acc: 0.4028
Epoch 2/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.9528 - acc: 0.5666 - val_loss: 0.8627 - val_acc: 0.6515
Epoch 3/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.7216 - acc: 0.7016 - val_loss: 0.7942 - val_acc: 0.6885
Epoch 4/30
7352/7352 [==============================] - 89s 12ms/step - loss: 0.4045 - acc: 0.8489 - val_loss: 0.6017 - val_acc: 0.7940
Epoch 5/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.2637 - acc: 0.9044 - val_loss: 0.3753 - val_acc: 0.8721
Epoch 6/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.2280 - acc: 0.9169 - val_loss: 0.4190 - val_acc: 0.8792
Epoch 7/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.2080 - acc: 0.9271 - val_loss: 0.3075 - val_acc: 0.9006


In [0]:
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
print("===================================================================")
score

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 508        1  ...                   0                 3
SITTING                  0      415  ...                   0                 0
STANDING                 0      104  ...                   0                 0
WALKING                  0        3  ...                  25                 0
WALKING_DOWNSTAIRS       0        4  ...                 401                 9
WALKING_UPSTAIRS         0       25  ...                   1               436

[6 rows x 6 columns]
2947/2947 [==============================] - 4s 1ms/step


[0.4257095748275667, 0.9009161859518154]

In [27]:
model = Sequential()
# Configuring the parameters
model.add(LSTM(32, input_shape=(timesteps, input_dim),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
dropout_8 (Dropout)          (None, 128, 32)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 198       
Total params: 13,894
Trainable params: 13,894
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 64s 9ms/step - loss: 1.3137 - acc: 0.4412 - val_loss: 1.2738 - val_acc: 0.4364
Epoch 2/30
7352/7352 [==============================] - 61s 8ms/step - loss: 0.9880 - acc: 0.5638 - val_loss: 0.8160 - val_acc: 0.5918
Epoch 3/30
7352/7352 [==============================] - 61s 8ms/step - loss: 0.8551 - acc: 0.5989 - val_loss: 0.7576 - val_acc: 0.5948
Epoch 4/30
7352/7352 [==============================] - 60s 8ms/step - loss: 1.0227 - acc: 0.5306 - val_loss: 1.3627 - val_acc: 0.3763
Epoch 5/30
7352/7352 [==============================] - 60s 8ms/step - loss: 1.3331 - acc: 0.4068 - val_loss: 1.3900 - val_acc: 0.3414
Epoch 6/30
7352/7352 [==============================] - 60s 8ms/step - loss: 1.2472 - acc: 0.4066 - val_loss: 1.0449 - val_acc: 0.4869
Epoch 7/30
7352/7352 [==============================] - 61s 8ms/step - loss: 0.9513 - acc: 0.5238 - val_loss: 0.9030 - val_acc: 0.5219
Epoch 8

In [29]:
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
print("===================================================================")
score

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                  5      414  ...                   1                 6
STANDING                 0      117  ...                   0                10
WALKING                  0        0  ...                  29                 3
WALKING_DOWNSTAIRS       0        0  ...                 417                 2
WALKING_UPSTAIRS         0        0  ...                   7               433

[6 rows x 6 columns]
2947/2947 [==============================] - 2s 675us/step


[0.330357293030449, 0.9032914828639295]

In [31]:
model = Sequential()
# Configuring the parameters
model.add(LSTM(128, input_shape=(timesteps, input_dim),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 128, 128)          70656     
_________________________________________________________________
dropout_12 (Dropout)         (None, 128, 128)          0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 390       
Total params: 120,454
Trainable params: 120,454
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train,Y_train,batch_size=batch_size,validation_data=(X_test, Y_test),epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 105s 14ms/step - loss: 1.2247 - acc: 0.4691 - val_loss: 1.7625 - val_acc: 0.1649
Epoch 2/30
7352/7352 [==============================] - 101s 14ms/step - loss: 1.4690 - acc: 0.3660 - val_loss: 1.3499 - val_acc: 0.3977
Epoch 3/30
7352/7352 [==============================] - 100s 14ms/step - loss: 1.3138 - acc: 0.4331 - val_loss: 1.4018 - val_acc: 0.3573
Epoch 4/30
7352/7352 [==============================] - 101s 14ms/step - loss: 1.3620 - acc: 0.3928 - val_loss: 1.4978 - val_acc: 0.2986
Epoch 5/30
7352/7352 [==============================] - 103s 14ms/step - loss: 1.3831 - acc: 0.3686 - val_loss: 1.3716 - val_acc: 0.3573
Epoch 6/30
7352/7352 [==============================] - 103s 14ms/step - loss: 1.3121 - acc: 0.4225 - val_loss: 1.3491 - val_acc: 0.3984
Epoch 7/30
7352/7352 [==============================] - 102s 14ms/step - loss: 1.2424 - acc: 0.4434 - val_loss: 1.3086 - val_acc: 

In [33]:
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
print("===================================================================")
score

Pred                LAYING  SITTING  STANDING  WALKING
True                                                  
LAYING                 493        0        27       17
SITTING                  0      243       248        0
STANDING                 0       20       508        4
WALKING                  0        0        24      472
WALKING_DOWNSTAIRS       0        0         4      416
WALKING_UPSTAIRS         0        0         8      463
2947/2947 [==============================] - 4s 1ms/step


[0.8343164877119734, 0.5822870715982355]

In [1]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = [" # hidden layers","# neurons", "loss", "Accuracy"]

x.add_row([1,32 , 0.4753, 0.9039 ])
x.add_row([1,128 , 0.4440, 0.8768 ])
x.add_row([1,64, 0.4244, 0.8941])
x.add_row([1,200, 0.4257, 0.9009])
x.add_row([2,"first lstm layer =32, second layer = 32", 0.3303, 0.9032])
x.add_row([2,"first lstm layer =128, second layer = 64", 0.8343, 0.5822])

print(x)

+------------------+------------------------------------------+--------+----------+
|  # hidden layers |                # neurons                 |  loss  | Accuracy |
+------------------+------------------------------------------+--------+----------+
|        1         |                    32                    | 0.4753 |  0.9039  |
|        1         |                   128                    | 0.444  |  0.8768  |
|        1         |                    64                    | 0.4244 |  0.8941  |
|        1         |                   200                    | 0.4257 |  0.9009  |
|        2         | first lstm layer =32, second layer = 32  | 0.3303 |  0.9032  |
|        2         | first lstm layer =128, second layer = 64 | 0.8343 |  0.5822  |
+------------------+------------------------------------------+--------+----------+


Conclusion:
- Running into the problem of exploding gradients while using relu or tanh as activation function 
- Best scores are for 2 lstm layer architecture with each layer containing 32 neurons and 2 layers of dropout
-As we increase number of neurons in 2 lstm layer architecture, performance tends to reduce
-Most of the single lstm layer network gave comparable results when used dropouts